In [18]:
import torch
import torch.nn as nn

In [19]:
class ResidualModule(nn.Module): # for resnet 18 and resnet 34
    def __init__(self, input_channel, output_channel, stride=1):
        super().__init__()
        self.module1 = nn.Conv2d(input_channel, output_channel, 3, stride, padding=1)
        self.module2 = nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.layers = nn.Sequential(
            self.module1,
            nn.BatchNorm2d(output_channel),
            nn.ReLU(inplace=True),
            self.module2,
            nn.BatchNorm2d(output_channel),
        )

        if (input_channel != output_channel):
            self.shortcut = nn.Sequential(
                nn.Conv2d(input_channel, output_channel, 1, stride),
                nn.BatchNorm2d(output_channel),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, X):
        return nn.ReLU(inplace=True)(self.shortcut(X) + self.layers(X))

In [20]:
class ResidualModuleBottleNeck(nn.Module): # for resnet >=50
    def __init__(self, input_channel, output_channel, stride=1):
        super().__init__()
        self.module1 = nn.Conv2d(input_channel, output_channel / 4, 1)
        self.module2 = nn.Conv2d(output_channel / 4, output_channel / 4, 3, stride, padding=1)
        self.module3 = nn.Conv2d(output_channel / 4, output_channel, 1)
        self.layers = nn.Sequential(
            self.module1,
            nn.BatchNorm2d(output_channel / 4),
            nn.ReLU(),
            self.module2,
            nn.BatchNorm2d(output_channel / 4),
            nn.ReLU(),
            self.module3,
            nn.BatchNorm2d(output_channel),
        )

        if (input_channel != output_channel):
            self.shortcut = nn.Sequential(
                nn.Conv2d(input_channel, output_channel, 1, stride),
                nn.BatchNorm2d(output_channel),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, X):
        return nn.ReLU(self.shortcut(X) + self.layers(X))

In [21]:
class ConvLayer(nn.Module):
    def __init__(self, residual_module_type, input_channel, output_channel, depth, first_stride=1):
        super().__init__()
        self.modules = [residual_module_type(input_channel, output_channel, first_stride)]
        for i in range(1, depth):
            self.modules.append(residual_module_type(output_channel, output_channel))
        self.layers = nn.ModuleList(self.modules)

    def forward(self, X):
        for module in self.layers:
            X = module(X)
        return X

In [22]:
class ResNet(nn.Module):
    def __init__(self, residual_module_type, residual_module_depth, num_classes=2):
        super().__init__()

        self.init_block = nn.Sequential(
            nn.Conv2d(3, 64, 7, 2, padding=3),
            nn.MaxPool2d(3, 2)
        )

        self.layer1 = ConvLayer(residual_module_type, 64, 64, residual_module_depth[0])
        self.layer2 = ConvLayer(residual_module_type, 64, 128, residual_module_depth[1], first_stride=2)
        self.layer3 = ConvLayer(residual_module_type, 128, 256, residual_module_depth[2], first_stride=2)
        self.layer4 = ConvLayer(residual_module_type, 256, 512, residual_module_depth[3], first_stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Linear(512, num_classes),
            nn.Softmax(dim=1),
        )

    def forward(self, X):
        X = self.init_block(X)
        X = self.layer1(X)
        X = self.layer2(X)
        X = self.layer3(X)
        X = self.layer4(X)
        X = self.avg_pool(X)
        X = X.view(X.size(0), -1)
        X = self.fc(X)
        return X

In [23]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

X = []
y = []
manipulated_path = './data/manipulated'
original_path = './data/original'
print("processing fake datasets")
for img_name in tqdm(os.listdir(manipulated_path)):
    X.append(np.array(Image.open(f'{manipulated_path}/{img_name}')))
    y.append([1, 0])
print("processing true datasets")
for img_name in tqdm(os.listdir(original_path)):
    X.append(np.array(Image.open(f'{original_path}/{img_name}')))
    y.append([0, 1])
X = np.array(X).swapaxes(3, 1).swapaxes(3, 2)
y = np.array(y)
print(X.shape)
print(y.shape)

np.random.seed(42)
indices = np.random.permutation(X.shape[0])
X_shuffled = []
y_shuffled = []
for i in range(X.shape[0]):
    X_shuffled.append(X[indices[i]])
    y_shuffled.append(y[indices[i]])
X_shuffled = np.array(X_shuffled, dtype=np.float32)
y_shuffled = np.array(y_shuffled, dtype=np.float32)
train_X = X_shuffled[:int(X.shape[0] * 0.8)]
train_y = y_shuffled[:int(X.shape[0] * 0.8)]
valid_X = X_shuffled[int(X.shape[0] * 0.8):int(X.shape[0] * 0.9)]
valid_y = y_shuffled[int(X.shape[0] * 0.8):int(X.shape[0] * 0.9)]
test_X = X_shuffled[int(X.shape[0] * 0.9):]
test_y = y_shuffled[int(X.shape[0] * 0.9):]
print(train_X.shape[0])
print(valid_X.shape[0])
print(test_X.shape[0])

processing fake datasets


100%|██████████| 8000/8000 [00:20<00:00, 399.18it/s]


processing true datasets


100%|██████████| 4000/4000 [00:09<00:00, 409.24it/s]


(12000, 3, 299, 299)
(12000, 2)
9600
1200
1200


In [24]:
import math

def process_epoch(X, y, model, opt=None, batch_size=1):
    if opt:
        model.train()
    else:
        model.eval()

    loss_func = nn.CrossEntropyLoss()
    correct, loss_sum, n_step, n_samples = 0, 0, 0, 0

    iter = math.ceil(X.shape[0] / batch_size)
    for batch in tqdm(range(iter)):
        X_batch = X[batch * batch_size:min((batch + 1) * batch_size, X.shape[0])][:, :, :224, :224]
        y_batch = y[batch * batch_size:min((batch + 1) * batch_size, X.shape[0])]
        if opt:
            opt.zero_grad()
        pred_y = model(torch.tensor(X_batch).cuda())
        loss = loss_func(pred_y, torch.tensor(y_batch).cuda())
        correct += np.sum(np.array(pred_y.cpu().detach().numpy().argmax(axis=1) == y_batch.argmax(axis=1), dtype=np.int64))
        if opt:
            loss.backward()
            opt.step()
        loss_sum += loss.cpu().detach().numpy()
        n_step += 1
        n_samples += X_batch.shape[0]

    return 1 - correct / n_samples, loss_sum / n_step

def train(
        batch_size=100,
        epochs=10,
        optimizer=torch.optim.Adam,
        lr=0.001,
        weight_decay=0.001,
        model=None):

    if(model == None):
        print("Please choose a type of ResNet!")
        return

    opt = optimizer(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(epochs):
        print(f"Epoch {epoch}: ")
        train_acc, train_loss = process_epoch(train_X, train_y, model, batch_size=batch_size, opt=opt)
        test_acc, test_loss = process_epoch(test_X, test_y, model, batch_size=batch_size)
        print(f"Train Accuracy: {round(train_acc, 3)} Test Accuracy: {round(test_acc, 3)}")

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train(model=ResNet(ResidualModule, [3, 4, 6, 3]).to(device))

Epoch 0: 


100%|██████████| 12/12 [00:00<00:00, 13.72it/s]


Train Accuracy: 0.338 Test Accuracy: 0.322
Epoch 1: 


100%|██████████| 12/12 [00:00<00:00, 14.32it/s]


Train Accuracy: 0.332 Test Accuracy: 0.322
Epoch 2: 


...

Train Accuracy: 0.332 Test Accuracy: 0.322
Epoch 8: 


100%|██████████| 12/12 [00:00<00:00, 13.61it/s]


Train Accuracy: 0.332 Test Accuracy: 0.322
Epoch 9: 


100%|██████████| 12/12 [00:00<00:00, 13.87it/s]

Train Accuracy: 0.332 Test Accuracy: 0.322
